In [65]:
import pymongo
from pymongo import MongoClient, GEOSPHERE
import sys
sys.path.append('../src')
import os
import code4_data_ as f4
import pandas as pd
import numpy as np
import geopandas as gpd
import pymongo
from pymongo import MongoClient, GEOSPHERE

/usr/local/Caskroom/miniconda/base/envs/ironhack/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.9.1dev-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


## Establishing connection to mongoDB collection that match criteria with the offices selected

In [6]:
client = MongoClient()

In [19]:
berlin = client.ironhack.hqreq_berlin
paris = client.ironhack.hqreq_paris
amsterdam = client.ironhack.hqreq_amsterdam
companies2 = client.ironhack.companies2
companies1 = client.ironhack.companies1

## Connecting to collections with locations that matches criteria

## Establishing connection to mongoDB

In [8]:
df = f4.import_csv("../data_processed/gdf5_v1.csv")

In [10]:
import ast

In [11]:
df['geometry'] = df['geometry'].apply(lambda x: ast.literal_eval(x))

In [14]:
for each in df.geometry:
    each['coordinates'] = list(each['coordinates'])
df['geometry']

0     {'type': 'Point', 'coordinates': [13.4124563, ...
1     {'type': 'Point', 'coordinates': [13.4469031, ...
2     {'type': 'Point', 'coordinates': [13.38861, 52...
3     {'type': 'Point', 'coordinates': [2.3283853, 4...
4     {'type': 'Point', 'coordinates': [2.3797825, 4...
5     {'type': 'Point', 'coordinates': [2.288319, 48...
6     {'type': 'Point', 'coordinates': [2.3210737, 4...
7     {'type': 'Point', 'coordinates': [2.3698033, 4...
8     {'type': 'Point', 'coordinates': [2.2983373, 4...
9     {'type': 'Point', 'coordinates': [2.3304161, 4...
10    {'type': 'Point', 'coordinates': [2.324782, 48...
11    {'type': 'Point', 'coordinates': [4.8516964, 5...
12    {'type': 'Point', 'coordinates': [4.870312, 52...
13    {'type': 'Point', 'coordinates': [4.8909347, 5...
Name: geometry, dtype: object

In [43]:
dfdic = df.to_dict('records')

In [44]:
companies1.insert_many(dfdic) 

In [53]:
companies1.create_index([("geometry",GEOSPHERE)])

'geometry_2dsphere'

### What combination offices-venues is more succesfull?

#### Comparing selected offices form each city with venues matching criteria

In [68]:
comp = pd.DataFrame(companies1.find({}))

In [80]:
berlin_offices = comp.iloc[:3,7]

In [81]:
paris_offices = comp.iloc[3:11,7]

In [82]:
amsterdam_offices = comp.iloc[11:,7]

In [96]:
for office in berlin_offices:
    q1 = {"geometry":{ "$nearSphere":{"$geometry": office, "$maxDistance": 2000}}}
    berlin_general = list(berlin.find(q1))

In [97]:
for office in paris_offices:
    q1 = {"geometry":{ "$nearSphere":{"$geometry": office, "$maxDistance": 2000}}}
    paris_general = list(paris.find(q1))

In [98]:
for office in amsterdam_offices:
    q1 = {"geometry":{ "$nearSphere":{"$geometry": office, "$maxDistance": 2000}}}
    amsterdam_general = list(amsterdam.find(q1))

In [99]:
berlin_general

[{'_id': ObjectId('607e7ffeef33aa3ca4c3dbd1'),
  'requirement': 'Vegetarian / Vegan Restaurant',
  'name': 'Veggie Vega Trend',
  'lat': 52.48882852660237,
  'long': 13.395257592201233,
  'geometry': {'type': 'Point',
   'coordinates': [13.395257592201233, 52.48882852660237]}},
 {'_id': ObjectId('607e7ffeef33aa3ca4c3dbc6'),
  'requirement': 'Daycare',
  'name': 'Kita 1',
  'lat': 52.48587633513613,
  'long': 13.391434983073358,
  'geometry': {'type': 'Point',
   'coordinates': [13.391434983073358, 52.48587633513613]}},
 {'_id': ObjectId('607e7ffeef33aa3ca4c3dbcd'),
  'requirement': 'Jazz Club',
  'name': 'Yorckschlösschen',
  'lat': 52.493614967122426,
  'long': 13.38253056057901,
  'geometry': {'type': 'Point',
   'coordinates': [13.38253056057901, 52.493614967122426]}},
 {'_id': ObjectId('607e7ffeef33aa3ca4c3dbc9'),
  'requirement': 'Cocktail Bar',
  'name': 'Satellite Cocktail',
  'lat': 52.493619,
  'long': 13.394989,
  'geometry': {'type': 'Point', 'coordinates': [13.394989, 52.49

In [114]:
dfparis = pd.DataFrame(paris_general)
dfparis.to_csv('../data_processed/df_paris_matches.csv',index = False)

In [115]:
dfams = pd.DataFrame(amsterdam_general)
dfams.to_csv('../data_processed/df_amsterdam_matches.csv',index = False)

In [116]:
dfberl = pd.DataFrame(berlin_general)
dfberl.to_csv('../data_processed/df_berlin_matches.csv',index = False)

In [117]:
dfall = pd.concat([dfparis,dfams,dfberl],ignore_index = True)
dfall.to_csv('../data_processed/df_3citiesjoin_matches.csv',index = False)